<a href="https://colab.research.google.com/github/kiran-lenka/Bitcoin-Price-Predictor/blob/main/BitCoin_Major_Project_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing and Installing Requiremnts

In [ ]:
!pip install -q praw

# importing the necessary libraries
import warnings
warnings.filterwarnings("ignore")

import praw
import tweepy as tw


import pandas as pd
import numpy as np
import ssl  
import os
from datetime import date
from io import StringIO
import json

import nltk 
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Bitcoin Market Sentiment Analysis and Prediction

## Calculating public sentiment on Bitcoin

### Reddit Sentiment Analysis

In [ ]:
# reddit credentials
reddit = praw.Reddit(client_id = 'UYBiraXAwH8bcw',
                     client_secret = 'RMg2VFM9ncuAwLl61YB301SBfTZkUQ',
                     user_agent = 'MyAPI/0.0.1',
                     check_for_async=False
                    )

# getting posts from the subreddits
lst_reddit = []

# bitcoin subreddit
subreddit = reddit.subreddit('bitcoin')
# hot posts
for post in subreddit.hot(limit=1000):
  lst_reddit.append(post.title)
# new posts
for post in subreddit.new(limit=1000):
  lst_reddit.append(post.title)

# CryptoCurrency subreddit
subreddit = reddit.subreddit('CryptoCurrency')
# hot posts
for post in subreddit.hot(limit=1000):
  lst_reddit.append(post.title)
# new posts
for post in subreddit.new(limit=1000):
  lst_reddit.append(post.title)

# btc subreddit
subreddit = reddit.subreddit('btc')
# hot posts
for post in subreddit.hot(limit=1000):
  lst_reddit.append(post.title)
# new posts
for post in subreddit.new(limit=1000):
  lst_reddit.append(post.title)

# Crypto_General subreddit
subreddit = reddit.subreddit('Crypto_General')
# hot posts
for post in subreddit.hot(limit=1000):
  lst_reddit.append(post.title)
# new posts
for post in subreddit.new(limit=1000):
  lst_reddit.append(post.title)

# Coinbase subreddit
subreddit = reddit.subreddit('Coinbase')
# hot posts
for post in subreddit.hot(limit=1000):
  lst_reddit.append(post.title)
# new posts
for post in subreddit.new(limit=1000):
  lst_reddit.append(post.title)

# Binance subreddit
subreddit = reddit.subreddit('Binance')
# hot posts
for post in subreddit.hot(limit=1000):
  lst_reddit.append(post.title)
# new posts
for post in subreddit.new(limit=1000):
  lst_reddit.append(post.title)
 
# converting the list into a dataframe and displaying it 
df_reddit = pd.DataFrame(lst_reddit, columns=['Post Titles'])

In [ ]:
print("No. of rows (Reddit posts) in the dataset:",len(df_reddit))
df_reddit.head()

No. of rows (Reddit posts) in the dataset: 8030


,Post Titles
0,"Mentor Monday, April 25, 2022: Ask all your bi..."
1,"Daily Discussion, April 25, 2022"
2,"BREAKING: The Central African Republic, became..."
3,Mt. Hodl: 64% of all bitcoin has not moved in ...
4,At the stoplight in front of me.


In [ ]:
# classifying the post as positive, negative or neutral and displaying the results
sia = SIA()
results = []
for line in lst_reddit:
  pol_score = sia.polarity_scores(line)
  pol_score['Post Titles'] = line
  results.append(pol_score)

df_reddit_nlp = pd.DataFrame(results)

# compound is taken as the deciding factor is classifying the sentiment

# positive
df_reddit_nlp.loc[df_reddit_nlp['compound'] > 0.05, 'Sentiment'] = '1'

# negative
df_reddit_nlp.loc[df_reddit_nlp['compound'] < -0.05, 'Sentiment'] = '-1'

# neutral
df_reddit_nlp.loc[(df_reddit_nlp['compound'] >= -0.05) & (df_reddit_nlp['compound'] <= 0.05), 'Sentiment'] = '0'

In [ ]:
df_reddit_nlp.head()

,neg,neu,pos,compound,Post Titles,Sentiment
0,0.0,1.000,0.000,0.000,"Mentor Monday, April 25, 2022: Ask all your bi...",0
1,0.0,1.000,0.000,0.000,"Daily Discussion, April 25, 2022",0
2,0.0,0.903,0.097,0.128,"BREAKING: The Central African Republic, became...",1
3,0.0,1.000,0.000,0.000,Mt. Hodl: 64% of all bitcoin has not moved in ...,0
4,0.0,1.000,0.000,0.000,At the stoplight in front of me.,0


In [ ]:
df_reddit_nlp.to_csv('reddit-senti.csv')

### Twitter Sentiment Analysis

In [ ]:
%%time
consumer_key= 'uLPC3KfMtGFcEeq4CxEOohZeg'
consumer_secret= 'tywsJRvcr2zz5ICg7bkadbSIIjhGFmAlOLjJECjPqMfaRuwc1T'
access_token= '1300465599823314944-VkC6tWnEUrbxTZ1wYpWIxbc8LQCPNL'
access_token_secret= 'DDiF0cmidxoQlT2rgEUCGkP4E2DI8PBwz6WMS5QL51zOG'
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

search_words = ['#crypto', '#bitcoin','#cryptocurrency','#bitcoins']
date_since = '2021-05-01'
tweet_text = []
date_time = []
location = []

for words in search_words:
  tweets = tw.Cursor(api.search,
              q=words,
              lang="en",
              since=date_since).items(2000)
  for tweet in tweets:
    str1 = tweet.text
    str2 = tweet.created_at
    str3 = tweet.user.location
    tweet_text.append(str1)
    date_time.append(str2)
    location.append(str3)

df_twitter = pd.DataFrame()
df_twitter['Tweets'] = tweet_text
df_twitter['Created at'] = date_time
df_twitter['Location'] = location

CPU times: user 26.5 s, sys: 2.62 s, total: 29.1 s
Wall time: 50min 1s


In [ ]:
print("No. of rows (tweets) in the dataset:",len(df_twitter))
df_twitter.head()

No. of rows (tweets) in the dataset: 8000


,Tweets,Created at,Location
0,RT @NFT_Holding: FREE #NFT GIVEAWAY!! 🎉\n\nRT ...,2022-04-25 09:52:39,
1,RT @ShibnobiNews: Hey #Shibnobi is a beautiful...,2022-04-25 09:52:39,"Hồ Chí Minh, Việt Nam"
2,RT @GyroCommunity: First GameFi on @PlatON_Net...,2022-04-25 09:52:39,Dirumah
3,RT @ICThanat0s: 📢NFT GIVEAWAY!📢\nWe've teamed ...,2022-04-25 09:52:39,Nigeria
4,@CryptoTownEU Good Project \n@TahsinAhamedNil\...,2022-04-25 09:52:38,


In [ ]:
sia = SIA()
results = []
for line in tweet_text:
  pol_score = sia.polarity_scores(line)
  pol_score['Tweets'] = line
  results.append(pol_score)

df_twitter_nlp = pd.DataFrame(results)

# compound is taken as the deciding factor is classifying the sentiment

# positive
df_twitter_nlp.loc[df_twitter_nlp['compound'] > 0.05, 'Sentiment'] = '1'

# negative
df_twitter_nlp.loc[df_twitter_nlp['compound'] < -0.05, 'Sentiment'] = '-1'

# neutral
df_twitter_nlp.loc[(df_twitter_nlp['compound'] <= 0.05) & (df_twitter_nlp['compound'] >= -0.05) , 'Sentiment'] = '0'

df_twitter_nlp['Created at'] = date_time
df_twitter_nlp['Location'] = location

In [ ]:
df_twitter_nlp.head()

,neg,neu,pos,compound,Tweets,Sentiment,Created at,Location
0,0.0,0.765,0.235,0.7104,RT @NFT_Holding: FREE #NFT GIVEAWAY!! 🎉\n\nRT ...,1,2022-04-25 09:52:39,
1,0.0,0.843,0.157,0.5994,RT @ShibnobiNews: Hey #Shibnobi is a beautiful...,1,2022-04-25 09:52:39,"Hồ Chí Minh, Việt Nam"
2,0.0,0.888,0.112,0.3400,RT @GyroCommunity: First GameFi on @PlatON_Net...,1,2022-04-25 09:52:39,Dirumah
3,0.0,1.000,0.000,0.0000,RT @ICThanat0s: 📢NFT GIVEAWAY!📢\nWe've teamed ...,0,2022-04-25 09:52:39,Nigeria
4,0.0,0.734,0.266,0.4404,@CryptoTownEU Good Project \n@TahsinAhamedNil\...,1,2022-04-25 09:52:38,


In [ ]:
len(df_twitter_nlp)

8000

In [ ]:
df_twitter_nlp.drop(columns = ['Location'])

,neg,neu,pos,compound,Tweets,Sentiment,Created at
0,0.00,0.765,0.235,0.7104,RT @NFT_Holding: FREE #NFT GIVEAWAY!! 🎉\n\nRT ...,1,2022-04-25 09:52:39
1,0.00,0.843,0.157,0.5994,RT @ShibnobiNews: Hey #Shibnobi is a beautiful...,1,2022-04-25 09:52:39
2,0.00,0.888,0.112,0.3400,RT @GyroCommunity: First GameFi on @PlatON_Net...,1,2022-04-25 09:52:39
3,0.00,1.000,0.000,0.0000,RT @ICThanat0s: 📢NFT GIVEAWAY!📢\nWe've teamed ...,0,2022-04-25 09:52:39
4,0.00,0.734,0.266,0.4404,@CryptoTownEU Good Project \n@TahsinAhamedNil\...,1,2022-04-25 09:52:38
...,...,...,...,...,...,...,...
7995,0.15,0.850,0.000,-0.3182,Crypto and Bitcoin KYC: Everything Critical Yo...,-1,2022-04-21 00:30:00
7996,0.00,1.000,0.000,0.0000,Here we go...\n\n#Bitcoin 43k is next from cur...,0,2022-04-21 00:26:08
7997,0.00,1.000,0.000,0.0000,#Bitcoin supply audit!\nTotal #Bitcoins mined ...,0,2022-04-21 00:22:06
7998,0.00,0.898,0.102,0.1779,@baalazamon @SolanaFndn @ProjectSerum @PythNet...,1,2022-04-21 00:21:39


In [ ]:
df_twitter_nlp.to_csv('twitter-senti.csv')